In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'
SEED = None

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

for _ in range(10000):
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    # print(seed)
    
    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    
    tm = TsetlinMachine(train_x.shape[1], 5)
    out_1 = tm.forward(shuffled_x)
    tm.update(shuffled_y)
    out_2 = tm.forward(shuffled_x)
    assert torch.equal(shuffled_y,out_2) # This fails because it is impossible to find a W in the first layer that generates a zero Y when the FULL_X is the entire input space

IndexError: index 5 is out of bounds for dimension 1 with size 5

In [14]:
failed_training = 0
for j in range(50000):

    SEED = None
    if SEED:
        random.seed(SEED)
        torch.manual_seed(SEED)
    else:
        seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
        random.seed(seed)
        torch.manual_seed(seed)

    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    tm = TsetlinMachine(train_x.shape[1], 5)
    for i in range(100):
        # batch_idxs = torch.randint(0, shuffled_x.shape[0], (2,)) # this doesnt work right now
        batch_idxs = torch.randperm(shuffled_x.shape[0])[:shuffled_x.shape[0]-1]
        batch_x = shuffled_x[batch_idxs]
        batch_y = shuffled_y[batch_idxs]
        out_1 = tm.forward(batch_x)
        tm.update(batch_y)
        out_2 = tm.forward(batch_x)
        assert torch.equal(batch_y,out_2)

    out_3 = tm.forward(train_x)
    if not torch.equal(train_y,out_3):
        failed_training += 1

failed_training

KeyboardInterrupt: 

In [15]:
failed_training = 0
for j in range(50000):

    SEED = None
    if SEED:
        random.seed(SEED)
        torch.manual_seed(SEED)
    else:
        seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
        random.seed(seed)
        torch.manual_seed(seed)

    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    tm = TsetlinMachine(train_x.shape[1], 5)
    for i in range(100):
        # batch_idxs = torch.randint(0, shuffled_x.shape[0], (2,)) # this doesnt work right now
        batch_idxs = torch.randperm(shuffled_x.shape[0])[:2]
        batch_x = shuffled_x[batch_idxs]
        batch_y = shuffled_y[batch_idxs]
        out_1 = tm.forward(batch_x)
        tm.update(batch_y)
        out_2 = tm.forward(batch_x)
        assert torch.equal(batch_y,out_2)

    out_3 = tm.forward(train_x)
    if not torch.equal(train_y,out_3):
        failed_training += 1

failed_training

28

In [ ]:
failed_training = 0
failed_tms = []
succeeded_tms = []
for _ in range(50000):
    SEED = None
    if SEED:
        random.seed(SEED)
        torch.manual_seed(SEED)
    else:
        seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
        random.seed(seed)
        torch.manual_seed(seed)

    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    tm = TsetlinMachine(train_x.shape[1], 5)
    for i in range(100):
        batch_idxs = torch.randint(0, shuffled_x.shape[0], (1,))
        batch_x = shuffled_x[batch_idxs]
        batch_y = shuffled_y[batch_idxs]
        out_1 = tm.forward(batch_x)
        tm.update(batch_y)
        out_2 = tm.forward(batch_x)
        assert torch.equal(batch_y,out_2)

    out_3 = tm.forward(train_x)
    if not torch.equal(train_y,out_3):
        failed_training += 1
        failed_tms.append(tm)
    else:
        succeeded_tms.append(tm)

failed_training

0

In [ ]:
SEED = 5659339910399965995
if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)

shuffled_idx = torch.randperm(train_x.shape[0])
shuffled_x = train_x[shuffled_idx]
shuffled_y = train_y[shuffled_idx]

tm = TsetlinMachine(train_x.shape[1], 5)
out_1 = tm.forward(shuffled_x)
tm.update(shuffled_y)
out_2 = tm.forward(shuffled_x)
assert torch.equal(shuffled_y,out_2)

AssertionError: 